In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset
from PIL import Image
import os
import torchvision.models as models
import time
import copy
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import random
from collections import defaultdict
import pandas as pd

import torch.nn.functional as F



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_indexes(arr, value):
    indexes = []
    for i in range(len(arr)):
        if arr[i] == value:
            indexes.append(i)
    return indexes

def get_length_per_class(dataloader, classes):
    class_counts = defaultdict(int)
    total = 0
    for batch in dataloader:
        _, labels = batch 
        labels = labels.numpy().tolist()
        for label in labels:
            class_counts[label] += 1
            total +=1

    class_counts = dict(sorted(class_counts.items()))
    for class_label, count in class_counts.items():
        print(f"Class {classes[class_label]}: {count} samples out of {total}")
def load_data(data_dir,
                           batch_size,
                           data_type,
                           noise_type,
                           noise_percentage,                           
                           transform,                           
                           data_percentage=1,
                           show_classes = False, random_seed=21):
    
    if noise_type == "None":
        noise_type = ""
        noise_percentage = ""
    else:
        noise_type = "/" + str(noise_type)
        noise_percentage = "/" + str(noise_percentage)
    path = data_dir + noise_type + "/" + data_type + noise_percentage
    print("path: ", path)
    dataset = ImageFolder(root=path, transform=transform)
    original_classes = dataset.classes 
    num_samples = len(dataset)
    indices = list(range(num_samples))

    labels = dataset.targets
    class_to_idx = dataset.class_to_idx
    needed_length = int(num_samples*data_percentage/100)
    expected_length_per_class = int(needed_length/len(original_classes))
    print(f"needed_length: {needed_length}, expected_length_per_class: {expected_length_per_class}")
    if data_percentage != 100:
        new_indices = []
        for key, value in class_to_idx.items():
            all_indixes_of_class = get_indexes(labels, value)
            new_indices.extend(all_indixes_of_class[:expected_length_per_class])
    else:
        new_indices = indices
    length_dataset = len(new_indices)
    print("length of final dataset:", length_dataset)

    
    # sampler = SubsetRandomSampler(new_indices)

    dataloader = DataLoader(dataset, sampler=new_indices, batch_size=batch_size)

    if show_classes:
        get_length_per_class(dataloader, original_classes)
        
    random.shuffle(new_indices)

   
    dataloader = DataLoader(dataset, sampler=new_indices, batch_size=batch_size)

    return dataloader, length_dataset, original_classes


def train_model(model, criterion, optimizer, scheduler, num_epochs=25, batch_show = 1792):
    since = time.time()
    valid_acc = []
    train_acc = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()  

            running_loss = 0.0
            running_corrects = 0
            l = 0

            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.clone().detach().to(device)
                labels = labels.clone().detach().to(device)

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    if not isinstance(outputs, torch.Tensor):
                        outputs = outputs.logits
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                l += len(inputs)
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            if phase == 'train':
                scheduler.step()
                train_acc.append(epoch_acc.item())
            else:
                valid_acc.append(epoch_acc.item())
                
            
            
            
            print('\n{} Loss: {:.4f} Acc: {:.4f}\n'.format(
                phase, epoch_loss, epoch_acc))

            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    
   

    accuracy_history = [train_acc, valid_acc]
    model.load_state_dict(best_model_wts)
    return model, best_acc.item(), accuracy_history
    
    





print('ready')

ready


In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4680, 0.4647, 0.3441], std=[0.2322, 0.2272, 0.2394]) 
])   

noise_type = "gaussian_noise"
noise_percentage = 10
data_percentage = 100
total_size = 21000

train_size = data_percentage*total_size/100
data_dir = '/kaggle/input/vegetableimages1/vegetable_images'

train_loader, train_size, classes = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "train",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

valid_loader, valid_size, _ = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "validation",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

valid_loader_with_noise, _, _ = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "validation",
                           noise_type = noise_type,
                           noise_percentage = noise_percentage,                           
                           transform = transform,                           
                           data_percentage=data_percentage)
dataloaders = {'train':  train_loader, 
               'val': valid_loader_with_noise
               }
dataloaders_with_noise = {'train':  train_loader, 
               'val': valid_loader_with_noise
               }


test_loader,test_size_, _ = load_data(data_dir = data_dir,
                           batch_size = 64,
                           data_type = "test",
                           noise_type = "gaussian_noise",
                           noise_percentage = noise_percentage,                           
                           transform = transform,                           
                           data_percentage=data_percentage)


test_loader_without_noise, _, _ = load_data(data_dir =data_dir,
                           batch_size = 64,
                           data_type = "test",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)
dataset_sizes = {'train':  train_size, 
        'val': valid_size,
        'test': test_size_
       }




path:  /kaggle/input/vegetableimages1/vegetable_images/train
needed_length: 15000, expected_length_per_class: 1000
length of final dataset: 15000
path:  /kaggle/input/vegetableimages1/vegetable_images/validation
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  /kaggle/input/vegetableimages1/vegetable_images/gaussian_noise/validation/10
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  /kaggle/input/vegetableimages1/vegetable_images/gaussian_noise/test/10
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  /kaggle/input/vegetableimages1/vegetable_images/test
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000


In [3]:
dataset_sizes

{'train': 15000, 'val': 3000, 'test': 3000}

In [4]:
def load_efficientnet():
    efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
    in_features = efficientnet._modules['classifier'][-1].in_features
    out_features = len(classes)
    efficientnet._modules['classifier'][-1] = nn.Linear(in_features, out_features, bias=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    efficientnet = efficientnet.to(device)
    for param in efficientnet.parameters():
        param.requires_grad = True
    return efficientnet
    
    
    

# Hyperparameter tuning 

In [5]:
criterion = nn.CrossEntropyLoss()
learning_rates = [0.005]
momentums = [0, 0.1]
steps = [3, 4]
gammas = [0.1,  0.5]
c = 0
columns = ['model', 'num_epochs', 'learning_rate', 'momentum', 'step', 'gamma', 'accuracy', 'history']
num_epochs = 10
ht_results = pd.DataFrame(columns = columns)
best_valid_score = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for lr in learning_rates:
    for m in momentums:
        for s in steps:
            for g in gammas: 
                c+=1
                print(f"STARTING {c} ITERATION")
                efficientnet_cr = load_efficientnet()
                print(f"Starting to test lr = {lr}, m = {m}, s = {s}, g = {g}")
                optimizer = optim.SGD(efficientnet_cr.parameters(), lr= lr, momentum = m)
                step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
                efficientnet_cr, valid_score, history = train_model(efficientnet_cr, criterion, optimizer, 
                                                      step_lr_scheduler, num_epochs=num_epochs)
                if best_valid_score < valid_score:
                    best_valid_score = valid_score
                    best_efficientnet = efficientnet_cr
                ht_results.loc[len(ht_results.index)] = ['efficientnet', num_epochs, lr, m, s, g, valid_score, history]                
                print(ht_results.loc[len(ht_results.index) - 1])
                
        
        



STARTING 1 ITERATION


/opt/conda/lib/python3.10/site-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to /root/.cache/torch/hub/torchhub.zip
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn

Starting to test lr = 0.005, m = 0, s = 3, g = 0.1
Epoch 1/10
----------

train Loss: 2.0560 Acc: 0.6849


val Loss: 1.4150 Acc: 0.9247

Epoch 2/10
----------

train Loss: 0.8940 Acc: 0.9495


val Loss: 0.6102 Acc: 0.9617

Epoch 3/10
----------

train Loss: 0.3937 Acc: 0.9708


val Loss: 0.3465 Acc: 0.9737

Epoch 4/10
----------

train Loss: 0.2240 Acc: 0.9803


val Loss: 0.2385 Acc: 0.9817

Epoch 5/10
----------

train Loss: 0.1503 Acc: 0.9873


val Loss: 0.1815 Acc: 0.9870

Epoch 6/10
----------

train Loss: 0.1235 Acc: 0.9901


val Loss: 0.1803 Acc: 0.9870

Epoch 7/10
----------

train Loss: 0.1197 Acc: 0.9903


val Loss: 0.1763 Acc: 0.9870

Epoch 8/10
----------

train Loss: 0.1166 Acc: 0.9905


val Loss: 0.1722 Acc: 0.9873

Epoch 9/10
----------

train Loss: 0.1132 Acc: 0.9901


val Loss: 0.1683 Acc: 0.9867

Epoch 10/10
----------

train Loss: 0.1101 Acc: 0.9911


val Loss: 0.1646 Acc: 0.9867

Training complete in 22m 55s
Best val Acc: 0.987333
model                               

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Starting to test lr = 0.005, m = 0, s = 3, g = 0.5
Epoch 1/10
----------

train Loss: 2.0518 Acc: 0.6863


val Loss: 1.4154 Acc: 0.9193

Epoch 2/10
----------

train Loss: 0.8980 Acc: 0.9455


val Loss: 0.6004 Acc: 0.9540

Epoch 3/10
----------

train Loss: 0.4017 Acc: 0.9698


val Loss: 0.3386 Acc: 0.9713

Epoch 4/10
----------

train Loss: 0.2280 Acc: 0.9815


val Loss: 0.2316 Acc: 0.9800

Epoch 5/10
----------

train Loss: 0.1516 Acc: 0.9879


val Loss: 0.1751 Acc: 0.9837

Epoch 6/10
----------

train Loss: 0.1239 Acc: 0.9905


val Loss: 0.1741 Acc: 0.9840

Epoch 7/10
----------

train Loss: 0.1205 Acc: 0.9906


val Loss: 0.1702 Acc: 0.9843

Epoch 8/10
----------

train Loss: 0.1170 Acc: 0.9913


val Loss: 0.1661 Acc: 0.9847

Epoch 9/10
----------

train Loss: 0.1128 Acc: 0.9917


val Loss: 0.1623 Acc: 0.9850

Epoch 10/10
----------

train Loss: 0.1101 Acc: 0.9909


val Loss: 0.1586 Acc: 0.9853

Training complete in 22m 29s
Best val Acc: 0.985333
model                               

...

Starting to test lr = 0.005, m = 0.1, s = 3, g = 0.5
Epoch 1/10
----------

train Loss: 1.9809 Acc: 0.7211


val Loss: 1.2940 Acc: 0.9443

Epoch 2/10
----------

train Loss: 0.7839 Acc: 0.9549


val Loss: 0.5275 Acc: 0.9630

Epoch 3/10
----------

train Loss: 0.3386 Acc: 0.9735


val Loss: 0.3060 Acc: 0.9727

Epoch 4/10
----------

train Loss: 0.1927 Acc: 0.9846


val Loss: 0.2138 Acc: 0.9810

Epoch 5/10
----------

train Loss: 0.1293 Acc: 0.9898


val Loss: 0.1639 Acc: 0.9827

Epoch 6/10
----------

train Loss: 0.1060 Acc: 0.9914


val Loss: 0.1628 Acc: 0.9827

Epoch 7/10
----------

train Loss: 0.1022 Acc: 0.9921


val Loss: 0.1593 Acc: 0.9830

Epoch 8/10
----------

train Loss: 0.0996 Acc: 0.9923


val Loss: 0.1557 Acc: 0.9837

Epoch 9/10
----------

train Loss: 0.0969 Acc: 0.9923


val Loss: 0.1523 Acc: 0.9837

Epoch 10/10
----------

train Loss: 0.0945 Acc: 0.9925


val Loss: 0.1490 Acc: 0.9837

Training complete in 22m 1s
Best val Acc: 0.983667
model                              

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Starting to test lr = 0.005, m = 0.1, s = 4, g = 0.1
Epoch 1/10
----------

train Loss: 1.9861 Acc: 0.7102


val Loss: 1.2871 Acc: 0.9313

Epoch 2/10
----------

train Loss: 0.7803 Acc: 0.9525


val Loss: 0.5227 Acc: 0.9627

Epoch 3/10
----------

train Loss: 0.3362 Acc: 0.9751


val Loss: 0.2991 Acc: 0.9737

Epoch 4/10
----------

train Loss: 0.1918 Acc: 0.9840


val Loss: 0.2072 Acc: 0.9803

Epoch 5/10
----------

train Loss: 0.1282 Acc: 0.9893


val Loss: 0.1578 Acc: 0.9850

Epoch 6/10
----------

train Loss: 0.1057 Acc: 0.9915


val Loss: 0.1562 Acc: 0.9853

Epoch 7/10
----------

train Loss: 0.1029 Acc: 0.9915


val Loss: 0.1528 Acc: 0.9860

Epoch 8/10
----------

train Loss: 0.0998 Acc: 0.9920


val Loss: 0.1493 Acc: 0.9867

Epoch 9/10
----------

train Loss: 0.0968 Acc: 0.9921


val Loss: 0.1460 Acc: 0.9873

Epoch 10/10
----------

train Loss: 0.0944 Acc: 0.9924


val Loss: 0.1428 Acc: 0.9873

Training complete in 22m 6s
Best val Acc: 0.987333
model                              

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Epoch 1/10
----------

train Loss: 1.9767 Acc: 0.7253


val Loss: 1.2749 Acc: 0.9353

Epoch 2/10
----------

train Loss: 0.7686 Acc: 0.9558


val Loss: 0.5124 Acc: 0.9637

Epoch 3/10
----------

train Loss: 0.3300 Acc: 0.9749


val Loss: 0.2959 Acc: 0.9737

Epoch 4/10
----------

train Loss: 0.1894 Acc: 0.9839


val Loss: 0.2063 Acc: 0.9833

Epoch 5/10
----------

train Loss: 0.1274 Acc: 0.9897


val Loss: 0.1579 Acc: 0.9850

Epoch 6/10
----------

train Loss: 0.1045 Acc: 0.9909


val Loss: 0.1569 Acc: 0.9857

Epoch 7/10
----------

train Loss: 0.1016 Acc: 0.9921


val Loss: 0.1535 Acc: 0.9860

Epoch 8/10
----------

train Loss: 0.0985 Acc: 0.9921


val Loss: 0.1501 Acc: 0.9860

Epoch 9/10
----------

train Loss: 0.0961 Acc: 0.9921


val Loss: 0.1468 Acc: 0.9863

Epoch 10/10
----------

train Loss: 0.0935 Acc: 0.9931


val Loss: 0.1436 Acc: 0.9863

Training complete in 22m 11s
Best val Acc: 0.986333
model                                                 efficientnet
num_epochs          

In [6]:
ht_results.to_csv('efficientnet_results.csv', index=False)

In [7]:
torch.save(best_efficientnet, "best_efficientnet.pth")

In [8]:
!zip -r file.zip /kaggle/working

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/efficientnet_results.csv (deflated 82%)
  adding: kaggle/working/best_efficientnet.pth (deflated 8%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)


In [9]:
!ls

best_efficientnet.pth  efficientnet_results.csv  file.zip


In [10]:
from IPython.display import FileLink
FileLink(r'file.zip')

/kaggle/working/file.zip

In [11]:
best_valid_score

0.988